# <center>Informatique tc3 (Projet Web) - TD1</center>

## <center style="color: #66d">Récupération, mise en forme, et stockage de données via Python / SQLite</center>

Dans le cadre de ce TD nous récupérerons en ligne des données concernant des pays, que nous mettrons en forme et stockerons dans une base de données afin de les utiliser pour obtenir des représentations graphiques, tout cela à l'aide de Python.

<h3 style="background-color:#eef; padding:0.5em">1. Contexte et mise en place des outils

La page wikipédia d'un pays (cf. par exemple https://en.wikipedia.org/wiki/Zimbabwe) donne de nombreuses informations, souvent peu structurées (texte de la page) mais dont certaines le sont suffisament pour pouvoir donner lieu à un traitement automatique (cf. boîte d'informations située à droite de la page) :
<img src="infobox.png">
<div style="text-align:center; font-size:80%; margin-top:0.5em">Partie supérieure de la boîte d'information relative au Zimbabwe</div>

On peut ainsi espérer récupérer des informations comme le nom officiel du pays, un lien vers une image du drapeau, une devise, un lien vers l'hymne national, le nom et les coordonnées géographiques de la capitale, le nombre d'habitants, *etc.*

L'objectif étant de réaliser une application délivrant des informations sur de nombreux pays, il n'est pas envisageable d'interrroger wikipédia à chaque fois. La construction de notre application passera donc par une phase d'initialisation lors de laquelle les informations nécessaires seront récupérées, puis stockées dans une base de données pour être réutilisées par la suite.

Le système de gestion de base de données qui sera utilisé pour la suite de ce TD et pour votre projet est SQLite. Ce système très simple fonctionne en stockant une base de données dans un fichier unique au format <code>.sqlite</code>.

<b>1.1. Analyse des pages Wikipédia :</b>
    
Pour récupérer les informations qui nous intéressent, il n'est pas question d'aspirer le contenu des  pages au format html, car d'une page Wikipédia à l'autre, html n'est pas utilisé de manière suffisamment structurée pour y retrouver les données.

On peut alors penser au format wiki, qui correspond au code source tell qu'il est édité par les auteurs. On peut en avoir un aperçu à l'adresse : https://en.wikipedia.org/w/index.php?title=Zimbabwe&action=edit. Attention toutefois à ne pas en profiter pour modifier la page.

Ce format est plus structurant et devrait pouvoir être analysé pour en extraire les informations...

Pensons-nous être les premiers à nous poser cette question ? Sans doute pas :<br>
cf. https://stackoverflow.com/questions/3312346/how-to-get-the-infobox-data-from-wikipedia

Il existe différentes solutions pour récupérer les informations que nous recherchons, comme par exemple <a href="https://www.wikidata.org/wiki/Wikidata:Introduction/fr">wikidata</a>, une base de données en ligne, libre, collaborative, et multilingue, qui collecte des données structurées pour alimenter entre autres Wikipédia et Wikimedia Commons. Toutefois, pour des raisons à la fois pédagogiques et de simplicité, nous nous bornerons à analyser le contenu des pages wikipédia, technique connue sous le nom de <a href="https://en.wikipedia.org/wiki/Web_scraping">Web scraping</a>.

Nous utiliserons pour cela le module <code>wptools</code> disponible sur Github.
Si votre poste est correctment configuré avec la distribution Anaconda, le moyen le plus simple de l'installer est de lancer la commande suivante dans une fenêtre terminal (en ajoutant éventuellement l'option <tt>--user</tt> en cas d'échec) :

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 1 - </b> Installer <code>wptools</code>, puis, après avoir redémarré le noyau du notebook (cf. icône <img src="restart_kernel.png" style="display:inline; position:relative; top:-10px">), exécuter la cellule suivante qui ne devrait pas produire d'erreur&nbsp;:

In [11]:
#
# import du module d'analyse des pages wikipédia
#
import wptools

<b>1.2. Récupération d'information depuis une page Wikipédia :</b>

Premier essai :

In [12]:
page = wptools.page('Zimbabwe')
page.get_parse(False)
page.data['infobox']

en.wikipedia.org (parse) Zimbabwe


{'conventional_long_name': 'Republic of Zimbabwe',
 'common_name': 'Zimbabwe',
 'native_name': '{{collapsible list\n| titlestyle |=| background:transparent;text-align:center;line-height:normal;font-size:84%;\n| title |=| 13 other official names\n| |Infobox|subbox|=|yes|bodystyle|=|font-size:77%;font-weight:normal;\n| rowclass1 |=| mergedrow | label1 |=| [[Shona language|Shona]]:\n| data1 |=| |lang|sn|Nyika yeZimbabwe|\n| rowclass2 |=| mergedrow | label2 |=| [[Northern Ndebele language|Ndebele]]:\n| data2 |=| |lang|nd|Ilizwe leZimbabwe|\n| rowclass3 |=| mergedrow | label3 |=| [[Chewa language|Chewa]]:\n| data3 |=| |lang|ny|Dziko la Zimbabwe|\n| rowclass4 |=| mergedrow | label4 |=| [[Sena language|Chibarwe]]:\n| data4 |=| |lang|bwg|Dziko la Zimbabwe|\n| rowclass5 |=| mergedrow | label5 |=| [[Kalanga language|Kalanga]]:\n| data5 |=| |lang|kck|Hango yeZimbabwe|\n| rowclass6 |=| mergedrow | label6 |=| [[Tshwa language|Khoisan]]:\n| data6 |=| |lang|st|Zimbabwe Nù|\n| rowclass7 |=| mergedrow 

Remarques :
* La commande <code>wptools.page()</code> effectue une requête HTTP vers la page wikipédia dont on lui passe le nom.
* Le code ci-dessus demande ensuite d'analyser le contenu de la page, puis de récupérer le contenu de la boîte d'information.
* <code>page.data['infobox']</code> est un dictionnaire. Charge à nous maintenant de récupérer les informations qui nous intéressent.

<b>N.B.1</b> <code>wptools</code> émet des messages d'information (sur fond rose dans le notebook), qu'on pourra empêcher plus tard.<br>
<b>N.B.2</b> La documentation de <code>wptools</code> est consultable à l'adresse https://github.com/siznax/wptools


<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 2 - </b> A partir du code fourni, écrire une fonction <code>get_info(country)</code> qui prend en argument le nom de la page wikipédia du pays à analyser et renvoie la boîte d'information obtenue.

In [58]:
from zipfile import ZipFile
import json

def get_info(country):
    with ZipFile('africa.zip','r') as z:
    
        # liste des documents contenus dans le fichier zip
        #print(z.namelist())
        #print()
    
        # infobox de l'un des pays
        info = json.loads(z.read(country+".json"))
        #print(info)
        return info

<div style="background-color:#fdd;padding:10px;border-radius:3px">
<b>Remarque importante - </b> Si vous avez du mal à installer wp_tools, sachez qu'on l'introduit ici pour expliquer la démarche. Par contre vous n'en aurez rapidement plus besoin dans la suite puisqu'on vous fournira les données afin de vous éviter de faire de trop fréquentes requêtes sur wikipédia durant le déroulement du TD et du projet.

Pour contourner wp_tools, développez la fonction <code>get_info</code> à partir du code permettant de lire un fichier zip donné en question 3.2. De cette manière vous accèderez aux données via le fichier zip fourni, plutôt qu'en envoyant systématiquement des requêtes à wikipédia. Toute la suite exploite le résultat de cette fonction, et ne posera donc pas de problème en l'absence de wptools.

In [59]:
get_info('Zimbabwe')

{'conventional_long_name': 'Republic of Zimbabwe',
 'common_name': 'Zimbabwe',
 'native_name': '{{collapsible list\n  | titlestyle |=| background:transparent;text-align:center;line-height:normal;font-size:84%;\n  | title |=| 13 other official names\n  | |Infobox|subbox|=|yes|bodystyle|=|font-size:77%;font-weight:normal;\n   | rowclass1 |=| mergedrow | label1 |=| [[Shona language|Shona]]:\n   | data1 |=| |lang|sn|Nyika yeZimbabwe|\n   | rowclass2 |=| mergedrow | label2 |=| [[Northern Ndebele language|Ndebele]]:\n   | data2 |=| |lang|nd|Ilizwe leZimbabwe|\n   | rowclass3 |=| mergedrow | label3 |=| [[Chewa language|Chewa]]:\n   | data3 |=| |lang|ny|Dziko la Zimbabwe|\n   | rowclass4 |=| mergedrow | label4 |=| [[Sena language|Chibarwe]]:\n   | data4 |=| |lang|bwg|Dziko la Zimbabwe|\n   | rowclass5 |=| mergedrow | label5 |=| [[Kalanga language|Kalanga]]:\n   | data5 |=| |lang|kck|Hango yeZimbabwe|\n   | rowclass6 |=| mergedrow | label6 |=| [[Tshwa language|Khoisan]]:\n   | data6 |=| |lang|s

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 3 - </b>  Ecrire une fonction <code>print_capital(info)</code> qui prend en argument la boîte d'information renvoyée par la fonction précédente, et affiche le nom du pays, la capitale et ses coordonnées géographiques *(peu importe le format pour l'instant)*.

In [62]:
def print_capital(info):
     print([info['common_name'],info['capital'],info['coordinates']])

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 4 - </b>  Tester ces fonctions avec quelques pays, par exemple le Zimbabwe, le Japon,  l'Allemagne, le Canada ou l'Australie.

In [63]:
print_capital(get_info('Zimbabwe'))

['Zimbabwe', '[[Harare]]', '{{Coord |17|50|S|31|3|E|type:city}}']


<b>1.3 Mise en forme des données</b>

Il ne vous aura pas échappé que le format des informations récupérées n'est pas optimal (crochets autour du nom de la capitale, présence parfois d'informations supplémentaires non désirées, coordonnées peu exploitables telles quelles). Pour certains pays, même la récupération du nom du pays n'est pas triviale.

Les <a href="https://docs.python.org/3/library/re.html">expressions régulières</a> sont un outil puissant permettant de récupérer des informations dans une chaîne de caractères au format connu mais complexe. Il s'agit d'un mécanisme présent dans de nombreux langages de programmation, permettant d'analyser des chaînes de caractères et d'en extraire de l'information. 

Le code ci-dessous montre comment récupérer le nom de la capitale, compris entre les doubles crochets <span style="color:#800"><i>(à condition bien sûr que vous ayez bien réalisé la fonction <tt>get_info()</tt> comme demandé ci-dessus)</i></span> :<code>

In [64]:
#
# import du module permettant d'exploiter des expressions régulières
#
import re

zw = get_info('Zimbabwe')
capital = zw['capital']
print('Chaîne brute : {}'.format(capital))

# Analyse de la chaîne brute en la comparant à [[*]], et en mémorisant le contenu des crochets
m = re.match("\[\[(\w+)\]\]", capital)

capital = m.group(1)
print('Capitale : {}'.format(capital))

Chaîne brute : [[Harare]]
Capitale : Harare


<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 5 - </b>  Ecrire une fonction <code>get_name(info)</code> qui prend en argument une infobox de pays et renvoie le nom officiel du pays. Cette information se trouve a priori dans le champ nommé <code>conventional_long_name</code>.

In [65]:
def get_name(info):
    if 'conventional_long_name' in info:
        name=info['conventional_long_name']
        m=re.match("([\w,-]+?)\s*{{",name)
        if m:
            name=m.group(1)
    m=re.match("{{.*\]([\w, -]+)}}",name)
    if m:
        name=m.group(1)
    return name

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 6 - </b>  Tester avec les pays suggérés plus haut.
</div>

In [66]:
get_name(get_info('Zimbabwe'))

'Republic of Zimbabwe'

<div style="background-color:#ded;padding:10px;border-radius:3px">
<b>N.B.</b> 
Pour certains pays <i>(Singapour, Vanuatu...)</i> le champ <code>conventional_long_name</code> n'est pas présent, pour d'autres l'information est plus complexe et donc plus difficile à extraire. Il est donc prévu que vous complétiez cette fonction (et les suivantes) au fur et à mesure des besoins, éventuellement mais pas nécessairement au cours de ce TD, sinon lors des TDs suivants ou de votre projet, pour arriver à traiter tous les pays qui vous concernent. N'hésitez pas pour cela à traiter les cas particuliers comme tels si nécessaire.
</div>

Allez donc au plus simple pour l'instant, mais prévoyez de compléter ce code par la suite.


<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 7 - </b>  Dans la même démarche, écrire une fonction <code>get_capital(info)</code> qui prend en argument une infobox de pays et renvoie une chaîne de caractères avec le nom de la capitale, et tester avec les pays déjà évoqués plus haut.

In [10]:
#
# Récupération de la capitale d'un pays depuis l'infobox wikipédia
#

In [11]:
# test de la fonction get_capital(info)

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 8 - </b>  Procéder de même avec les coordonnées géographiques. Il sera nécessaire de convertir les expressions en degrés, minutes, secondes, avec indication de la direction N/S et E/O, pour obtenir une information en degrés sous la forme d'un nombre décimal, positif ou négatif.

La fonction <code>get_coords(info)</code> devra renvoyer un dictionnaire avec deux éléments respectivement nommés <code>lat</code> et <code>lon</code>.

In [12]:
#
# Récupération des coordonnées de la capitale depuis l'infobox d'un pays
#

In [13]:
# test de la fonction get_coords(info)

<h3 style="background-color:#eef; padding:0.5em">2. Mise en place d'une base de données</h3>

<b>2.1. Création de la base :</b>

En Python, la création d'une base de données SQLite est particulièrement simple à effectuer :

In [14]:
#
# import du module d'accès à la base de données
#
import sqlite3

#
# Ouverture d'une connexion avec la base de données
#
conn = sqlite3.connect('pays.sqlite')

Ce code permet d'ouvrir une connexion avec la base de données des pays. Si elle n'existe pas, elle est créée !

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 9 - </b> Vérifier l'apparition d'un fichier nommé <code>pays.sqlite</code> dans le répertoire dans lequel se trouve le présent notebook.
</div>

<img src="pays_sqlite.png">

Toutes les opérations sur une base de données de ce type peuvent être effectuées en Python via les fonctions présentes au sein du module <code>sqlite3</code>.

Cependant, pour manipuler de manière interactive le contenu de la base <i>(créer, supprimer ou modifier des tables et des enregistrements, effectuer des requêtes SQL...)</i> il existe des outils adaptés.

L'outil retenu dans le cadre de ce cours s'appelle "DB Browser for SQLite". C'est un logiciel libre qui existe pour toutes les plateformes (Windows, MacOs, nombreuses distribution Linux et Unix...).

<div style="background-color:#dde;padding:10px;border-radius:3px;margin-top:1.33em">
<b>Etape 10 - </b>Télécharger et installer <a href="http://sqlitebrowser.org/">DB Browser for SQLite</a> en suivant les instructions d'installation en fonction de votre système d'exploitation.
</div>

Démarrer DB Browser for SQLite en ouvrant la base de données (vide pour l'instant) qui vient d'être créée.

<img src="empty_db.png" width="600px" style="box-shadow:3px 3px 3px #888">
<div style="text-align:center; font-size:80%; margin-top:0.5em">Interface de DB Browser for SQLite</div>

<b>2.2. Création d'une table</b>

Avant de pouvoir enregistrer dans la base de données les informations que nous sommes maintenant capables de récupérer depuis wikipédia, il est nécessaire de décrire leur format en créant une table.

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 11 - </b> Via l'interface de DB Browser for SQLite, créer une table nommée "countries" destinée à recevoir les informations extraites de wikipédia, avec les champs suivants (décrits ici en langage SQL) <b>en n'oubliant pas d'enregistrer les modifications</b> :

<img src="table_countries.png" style="width:500px;border:1px solid #ccc; box-shadow:3px 3px 3px #888">
<div style="text-align:center; font-size:80%; margin-top:0.5em">Visualisation de la table countries dans DB Browser for SQLite</div>

<b>2.3. Ecriture dans la base</b>

Via Python, l'écriture d'un enregistrement dans la base s'effectue en soumettant une requête SQL INSERT. En supposant que la variable zw contient l'infobox récupérée sur wikipédia au sujet du Zimbabwe, voici comment procéder :

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 12 - </b> En vous basant sur le code ci-dessus, créer une fonction <code>save_country(conn,country,info)</code> qui enregistre les information dans la base de données, et dont les arguments sont la connexion ouverte avec la base de données, le nom du pays et l'infobox récupérée sur wikipédia.

In [15]:
# votre code ici

Tester avec au moins un pays, et vérifier à l'aide de DB Browser que l'enregistrement a bien été effectué :

<img src="enregistrement_zw.png" style="width:500px; border:1px solid #ccc;box-shadow:3px 3px 3px #888"/>
<div style="text-align:center; font-size:80%; margin-top:0.5em">Visualisation d'un enregistrement dans DB Browser for SQLite</div>

In [16]:
# votre code ici

<div style="background-color:#dde;padding:10px;border-radius:3px">
<b>Etape 13 - </b> Créer une fonction <code>read_country(conn,country)</code> dont les arguments sont la connexion ouverte avec la base de données et le nom du pays, qui tente de lire les informations du pays dans la base de données et les renvoie, ou renvoie None si le pays n'existe pas dans la base.

In [17]:
# votre code ici

<h3 style="background-color:#efe; padding:0.5em">3. Pour aller plus loin...</h3>

... et progresser vers la réalisation de votre projet, il s'agit maintenant d'extraire les informations de la page wikipédia de chacun des pays du continent qui vous concerne, et de les insérer dans la base de données.

<b>3.1. Vous avez dit scraping ?</b>

Le scraping systématique de nombreuses pages peut potentiellement prendre du temps, et ce d'autant plus que le serveur de wikipédia semble devenir réticent à répondre lorsqu'on lui transmet des requêtes en rafale à une fréquence peu crédible pour un opérateur humain *(il se protège contre les attaques par déni de service DoS)*.

Pour cette raison, pour vous permettre d'être plus efficaces dans cette étape, et la fraîcheur des informations récupérées n'étant pas un critère fondamental pour votre application, nous vous fournissons un fichier zip par continent, contenant pour chacun des pays concernés un fichier au format json avec le contenu de l'infobox tel que récupéré sur la page wikipédia.

<img src="south_america_zip.png" width="400" style="border:1px solid #1883d7; box-shadow:3px 3px 3px #888">
<div style="text-align:center; font-size:80%; margin-top:0.5em">Contenu du fichier south_america.zip</div>

Le nom des fichiers json correspond au nom de la page wikipédia du pays, et au champ que nous avons nommé <code>wp</code> dans la base de données.

<b>Rappel :</b> Affectation des continents aux groupes de projet :
* projet A : Afrique
* projet B : Asie
* projet C : Europe
* projet D : Amérique du nord
* projet E : Amérique du sud et Océanie

Pour information, la liste des pays par continents peut également se trouver sur le site https://www.countries-ofthe-world.com/. 

<b>3.2. Initialisation de la base</b>

<div style="background-color:#ded;padding:10px;border-radius:3px">
<b>TODO - </b> Ecrire un programme qui parcourt le contenu du (ou des) fichier(s) zip qui vous concerne(nt) et écrit dans la base de données les informations extraites de l'infobox de chacun des pays à l'aide des fonctions précédemment développées.
</div>
<p>
Voici comment obtenir la liste des documents contenus dans un fichier zip et récupérer le contenu d'une infobox <i>(à vous de généraliser)</i> :
</p>

In [8]:
from zipfile import ZipFile
import json

with ZipFile('oceania.zip','r') as z:
    
    # liste des documents contenus dans le fichier zip
    print(z.namelist())
    print()
    
    # infobox de l'un des pays
    info = json.loads(z.read('Vanuatu.json'))
    print(info)

['Australia.json', 'Fiji.json', 'Kiribati.json', 'Marshall_Islands.json', 'Federated_States_of_Micronesia.json', 'Nauru.json', 'New_Zealand.json', 'Palau.json', 'Papua_New_Guinea.json', 'Samoa.json', 'Solomon_Islands.json', 'Tonga.json', 'Tuvalu.json', 'Vanuatu.json']

{'coordinates': '{{Coord|display|=|t}}', 'leader_name2': '[[Charlot Salwai]]', 'conventional_long_name': 'Republic of Vanuatu', 'native_name': '{{unbulleted list|item_style|=|font-size:88%; |native name|bi|Ripablik blong Vanuatu|italic|=|no| |native name|fr|République de Vanuatu|italic|=|no}} {{native name|bi|Ripablik blong Vanuatu|italic|=|no}} {{native name|fr|République de Vanuatu|italic|=|no}}', 'image_flag': 'Flag of Vanuatu (official).svg', 'image_coat': 'Coat of Arms of Vanuatu.svg', 'common_name': 'Vanuatu', 'symbol_type': 'Coat of arms', 'national_motto': '{{native phrase|bi|"Long God yumi stanap"|italics|=|off|nolink|=|on}} <br/> {{native phrase|fr|Nous nous tenons devant Dieu|italics|=|off|nolink|=|on}} <br/> 

In [19]:
#
# Initialisation des informations dans la base de données
#

<b>3.3. S'amuser avec le contenu de la base de données</b>

Maintenant que nous disposons des coordonnées d'un certain nombre de capitales, il est d'ores et déjà possible de construire des requêtes SQL pour répondre à des questions comme :

* Quel est le pays (avec sa capitale et ses coordonnées) dont la capitale se situe le plus au nord ?
* Quel est le pays (avec sa capitale et ses coordonnées) dont la capitale se situe le plus à l'est ?
* Quels sont les pays dont la capitale se situe plus au nord que celle d'un pays donné ?
* Quel est le pays dont la capitale se trouve le plus près de celle d'un pays donné ?
* ad libitum...

<div style="background-color:#ded;padding:10px;border-radius:3px">
<b>SUGGESTION - </b> Se prendre au jeu et imaginer les requêtes SQL permettant de répondre à certaines de ces questions...

In [20]:
# votre code ici

<b>3.4. Renseigner la base avec l'ensemble des informations nécessaires au fonctionnement de l'application rêvée</b>

Les informations actuellement collectées permettront de réaliser une application minimale.

Pour aller plus loin, il sera pertinent de réfléchir aux données que vous voulez récupérer et exploiter pour votre projet. Pour cela il sera utile d'effectuer les étapes suivantes :

<div style="background-color:#ded;padding:10px;border-radius:3px;margin-top:1.33em">
<b>TODO - Travail en groupe de projet </b>

<ul>
<li> analyser le contenu de l'infobox pays des pages wikipédia pour retenir les informations qui vous intéressent, et noter le nom de la clé de chacune de ces données dans l'infobox,

<li> sur le modèle de <code>get_name()</code> et <code>get\_capital()</code>, créer une fonction <code>get_\*()</code> pour chacune des informations retenues pour extraire les données correspondantes de l'infobox et les mettre en forme,

<li> modifier la table "countries" pour ajouter les champs nécessaires,

<li> modifier la fonction <code>save_country</code> <i>(pour remplir la base en repartant d'une base vide)</i> ou créer une fonction <code>update_country</code> <i>(afin de compléter les enregistrements existants dans la base)</i> pour tenir compte de tous les champs.
</ul>

<p>
<b>N.B.</b> Vous trouverez des informations plus détaillées sur cette démarche, et notamment une liste de clés exploitables dans l'ensemble des infoboxes, dans le notebook dédié au projet.
</div>

<p>
Après avoir par exemple créé un champ nommé "continent" dans la table "countries" :
<p>
<img src="table_countries_continent.png" style="width:500px;border:1px solid #ccc; box-shadow:3px 3px 3px #888">
<div style="text-align:center; font-size:80%; margin-top:0.5em">Table countries après ajout du champ continent</div>

... voici une fonction permettant d'ajouter le nom du continent à un enregistrement déjà présent dans la base :

In [21]:
#
# Mise à jour du continent d'un pays dans la base de données 
#
def update_country_continent(conn,country,continent):
    
    # préparation de la commande SQL
    c = conn.cursor()
    sql = 'UPDATE countries SET continent=? WHERE wp=?'

    # soumission de la commande (noter que le second argument est un tuple)
    c.execute(sql,(continent,country))
    conn.commit()
    


In [22]:
# votre code ici